# Word Embedding

*Este cuadernillo está basado parcialmente en material desarrollado por el Dr. Matthieu Vernier*

### Motivación: representar las dimensiones semánticas de cada palabra

1. I want an orange.
2. I want an apple.

- Los enfoques <i>bag of words</i> no tienen la capacidad de calcular que las frases 1 y 2 son muy similares porque no tienen una manera de representar que las palabras 'orange' y 'apple' comparten características (<i>features</i>) comunes.

- En ese contexto las palabras se representan como vectores "1-Hot". Por ejemplo, supongamos que tenemos un vocabulario de sólo cinco palabras: King, Queen, Man, Woman y Child. Se codificaría la palabra 'Queen' como:

<img src="img/word2vec1.png"/>

- Sería más interesante poder representar la semántica de cada palabra tomando en cuentas ciertas características. 

<img src="img/word2vec2.png"/>


### Definición

El concepto de **word embedding** se refiere a un conjunto de técnicas utilizadas para aprender representaciones matemáticas, tipicamente vectores, de cada palabra.

Una de las técnicas más populares es __Word2Vec__ propuesto por un equipo de investigación de Google en 2013 (Efficient Estimation of Word Representations in Vector Space [Mikolov et al., 2013]).

Alternativas populares son __GloVe__ (propuesta por la Universidad de Stanford en 2014) y __FastText__ (propuesta por Facebook en 2016), que extende Word2Vec para considerar de mejor manera las palabras con errores ortográficas.

### Algunas propiedades de los word embeddings

- Tener representaciones vectoriales de las palabras permite calcular "razonamiento" de tipo __King - Man + Woman = ?__ y llegar a un resultado cerca de __Queen__.

<img src="img/word2vec4.png"/>

- Tener representaciones vectoriales de las palabras permite realizar razonamientos analógicos de tipo __A es a B, lo que C es a ..__ . Este tipo de propiedades es muy útil para aplicaciones de _Question Answering_ por ejemplo. Las respuestas a las pregutas siguientes <i>¿Cuál es la capital de Chile?</i> o <i>¿Cuáles son los clubs de fútbol en Chile?</i> se pueden responder adicionando vectores.

<img src="img/word2vec6.png"/>

<img src="img/word2vec7.png"/>

<img src="img/word2vec8.png"/>

### ¿Cómo se aprenden los vectores? - Redes neuronales

El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="img/word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' si aparece la palabra 'pelota' en el contexto?

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>



Los métodos de word embedding se construyen en base a grandes corpus de documentos que permiten entrenar redes neuronales artificiales con diversas arquitecturas, de tal forma que se consideran **los pesos de la capa interna de la red** como la representación de cada palabra.

**Continuous bag-of-words (CBOW):** En este caso la tarea de la red neuronal es predecir una palabra dado su contexto.

Para el cálculo de la verosimilitud de una palabra  $w_n$ se considera:

$$w_n \sim softmax(\rho^T \alpha_{d_n})$$

donde $\rho \in M_{LxN}$ es la matriz que contiene en sus columnas el vector de embedding  de cada palabra del vocabulario y $\alpha_{d_n} \in R^L$ es el vector de contexto correspondiente a la suma de los vectores de embedding de las palabras en el contexto de la palabra $w_n$.

**Skip-gram (SG)** La red neuronal asociada tiene como tarea predecir las palabras del contexto de una palabra dada.


<img src="img/modelos_embedding.png" width=800 height=500>         


<img src="img/skip-gram.png" width=800 height=500>
*imagen proveniente de Tesis de Magíster de Pablo Badilla, U. de Chile*

Las funciones objetivos mas utilizadas para optimizar estas redes neuronales artificiales son:

- Softmax jerárquico 

- Negative sampling (ver https://arxiv.org/pdf/1402.3722.pdf)

La biblioteca **Word2Vec** implementa CBOW y SG


**Para profundizar:** https://arxiv.org/pdf/1301.3781.pdf

Otras arquitecturas de red y funciones objetivo definen otros métodos de word embedding:

- Glove (global vectors): considera el contexto global de cada palabra en la función de optimización, no sólo el contexto local


- Lexvec (Lexical vectors): incorpora la medida PPMI (positive point-wise mutual information) en la optimización de la arquitectura skip-gram.


- FastText: incorpora los n-gramas que componen una palabra.


- ConceptNet: considera la elaboración de un grafo de conocimiento (en base a diversas fuentes) con las palabras del vocabulario que se utiliza para el ajuste de la red neuronal.


**Limitaciones de modelos de word embedding**:

- Las redes pre-entrenadas para generar la representación vectorial dependen fuertemente de los corpus utilizados.

- Mayormente desarrollados en inglés. Algunas iniciativas en español.### Limitaciones de los word embeddings

- No permiten tomar en cuenta el orden entre las palabras.

Ejemplo: "Estamos aqui para trabajar y no jugar" vs. "Estamos aqui para jugar y no trabajar"

- No permiten tomar en cuenta que ciertas palabras cambian de significado según el contexto.

Ejemplo: "I lost my computer __mouse__"




**Para mejorar algunas de estas limitaciones:**

- Combinar Word Embedding con redes neuronales (convolucionales (CNN) o secuenciales (Transformers, BERT, GPT-3)) que toman en cuenta el orden entre las palabras y el contexto de las palabras.

- **Attention Word Embedding (AWE)** Incorpora mejoras en el proceso de optimización en CBOW de manera que el vector de contexto que se utiliza para definir la función objetivo no le de la misma importancia a todas las palabras del contexto, ajustando pesos distintos de las palabras en el contexto de la palabra a predecir mediante mecanismos de atención (matrices key y query). Existe también una versión AWE-S que considera cada palabra descompuesta en n-gramas.

## Embedding Topic Models (ETM)

(ver https://arxiv.org/pdf/1907.04907.pdf)

En este caso se propone el siguiente proceso generativo secuencial:


1. Por cada documento generar la distribución de tópicos de una **Logistic Normal**:

$$\theta_d \sim {\cal{LN}}(0,I_{KxK})$$
    
2. Por cada documento $d$ generar la palabra $n$ de la manera siguiente:

    a. Asignar un tópico:
    
    $$z_{d,n} \sim Mult(\theta_d)$$
    
    b. Generar la palabra:
    
    $$w_{d,n} \sim Mult(softmax(\rho^T \alpha_{z_{d,n}}))$$
    
donde $\rho \in M_{LxN}$ es la matriz que contiene en sus columnas el vector de embedding  de cada palabra del vocabulario y $\alpha_{z_{d_n}} \in R^L$ es el vector de embedding del tópico $z_{d,n}$.
        
<img src="img/ETM.png" width=600 height=200>            

Como en los modelos anteriores, las distribuciones a posteriori no tienen expresiones analíticas, por lo que se recurre a la Inferencia Variacional para aproximarlas.

Los resultados de ETM  mejoran sustancialmente el desempeño de LDA al aumentar el tamaño del vocabulario.


<img src="img/etm_vs-lda.png" width=400 height=400>       